In [5]:
require 'nn';
require 'hdf5';

In [6]:
function build_model(dwin, nchar, nclass, hid1, hid2)
    -- Model with skip layer from Bengio, standards parameters
    -- should be:
    -- dwin = 5
    -- hid1 = 30
    -- hid2 = 100

    -- To store the whole model
    local dnnlm = nn.Sequential()

    -- Layer to embedd (and put the words along the window into one vector)
    local LT = nn.Sequential()
    local LT_ = nn.LookupTable(nchar,hid1)
    LT:add(LT_)
    LT:add(nn.View(-1, hid1*dwin))

    dnnlm:add(LT)

    local concat = nn.ConcatTable()

    local lin_tanh = nn.Sequential()
    lin_tanh:add(nn.Linear(hid1*dwin,hid2))
    lin_tanh:add(nn.Tanh())

    local id = nn.Identity()

    concat:add(lin_tanh)
    concat:add(id)

    dnnlm:add(concat)
    dnnlm:add(nn.JoinTable(2))
    dnnlm:add(nn.Linear(hid1*dwin + hid2, nclass))
    dnnlm:add(nn.LogSoftMax())

    -- Loss
    local criterion = nn.ClassNLLCriterion()

    return dnnlm, criterion
end


function train_model(train_input, train_output, dnnlm, criterion, dwin, nclass, eta, nEpochs, batchSize)
    -- Train the model with a mini batch SGD
    -- standard parameters are
    -- nEpochs = 1
    -- batchSize = 32
    -- eta = 0.01

    -- To store the loss
    local av_L = 0

    -- Memory allocation
    local inputs_batch = torch.DoubleTensor(batchSize,dwin)
    local targets_batch = torch.DoubleTensor(batchSize)
    local outputs = torch.DoubleTensor(batchSize, nclass)
    local df_do = torch.DoubleTensor(batchSize, nclass)

    for i = 1, nEpochs do
        -- timing the epoch
        local timer = torch.Timer()

        av_L = 0
        
        -- max renorm of the lookup table
        dnnlm:get(1):get(1).weight:renorm(2,1,1)
        
        -- mini batch loop
        for t = 1, train_input:size(1), batchSize do
            -- Mini batch data
            local current_batch_size = math.min(batchSize,train_input:size(1)-t)
            inputs_batch:narrow(1,1,current_batch_size):copy(train_input:narrow(1,t,current_batch_size))
            targets_batch:narrow(1,1,current_batch_size):copy(train_output:narrow(1,t,current_batch_size))
            
            -- reset gradients
            dnnlm:zeroGradParameters()
            --gradParameters:zero()

            -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
            outputs:narrow(1,1,current_batch_size):copy(dnnlm:forward(inputs_batch:narrow(1,1,current_batch_size)))

            -- Average loss computation
            local f = criterion:forward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size))
            av_L = av_L +f

            -- Backward pass
            df_do:narrow(1,1,current_batch_size):copy(criterion:backward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size)))
            dnnlm:backward(inputs_batch:narrow(1,1,current_batch_size), df_do:narrow(1,1,current_batch_size))
            dnnlm:updateParameters(eta)
            
        end
            
        print('Epoch '..i..': '..timer:time().real)
        print('Average Loss: '..av_L/math.floor(train_input:size(1)/batchSize))
       
    end

end

In [7]:
myFile = hdf5.open('../data_preprocessed/6-grams.hdf5','r')
data = myFile:all()
myFile:close()

In [19]:
data

{
  input_data_test : LongTensor - size: 367519


  input_data_valid : LongTensor - size: 115720
  F_train : DoubleTensor - size: 65033x7


  input_data_train : LongTensor - size: 599909
  input_matrix_train : DoubleTensor - size: 599903x5
  input_data_valid_nospace : LongTensor - size: 95827


  output_matrix_train : DoubleTensor - size: 599903
}


In [18]:
Nwin = 5

train_input = data['input_matrix_train']
train_output = data['output_matrix_train']

valid_space = data['input_data_valid_nospace']
valid = data['input_data_valid_nospace']:clone()

test = data['input_data_test']:clone()

In [9]:
torch.manualSeed(1)

nnlm1, crit = build_model(Nwin, 49, 2, 20, 16)

train_model(train_input, train_output, nnlm1, crit, Nwin, 2, 0.01, 20, 10)

Epoch 1: 40.021296977997	
Average Loss: 0.28291292916549	


Epoch 2: 34.586709976196	
Average Loss: 0.22909422380413	


Epoch 3: 32.872662067413	
Average Loss: 0.20544086393986	


Epoch 4: 37.268335103989	
Average Loss: 0.19439121899764	


Epoch 5: 36.69793510437	
Average Loss: 0.18717225893088	


...

Epoch 16: 28.563301086426	
Average Loss: 0.15711771041754	


Epoch 17: 28.409915208817	
Average Loss: 0.15611926878796	


Epoch 18: 29.172394037247	
Average Loss: 0.15523507092617	


Epoch 19: 28.571200847626	
Average Loss: 0.15446284926575	


Epoch 20: 28.591543912888	
Average Loss: 0.15377656872611	


### Greedy:

In [10]:
it = 1
i = 1
nextpred = torch.Tensor(2)

print('here')
while it<data['input_data_valid_nospace']:size(1)-(Nwin-1) do
    if it % 50 == 0 then
        print( it..'/'..data['input_data_valid_nospace']:size(1))
    end
    it = it + 1
    nextpred:copy(nnlm1:forward(valid:narrow(1,i,Nwin)));
    m, argm = nextpred:max(1)

    if argm[1] == 2 then
        i = i + 1
    elseif argm[1] == 1 then 
        valid_ = torch.LongTensor(valid:size(1)+1)
        valid_:narrow(1,1,i+(Nwin-1)):copy(valid:narrow(1,1,i+(Nwin-1)))
        valid_[i+Nwin] = 1
        valid_:narrow(1,i+(Nwin-1)+2,valid:size(1)-i-(Nwin-1)):copy(valid:narrow(1,i+(Nwin-1)+1,valid:size(1)-i-(Nwin-1)))
        valid = valid_
        i = i + 2
    end 
end

here	


50/95827	


100/95827	


150/95827	


200/95827	


...

95600/95827	


95650/95827	


95700/95827	


95750/95827	


95800/95827	


In [14]:
num_sent = 0
for i = 5,valid_space:size(1) do
    if valid_space[i] == 2 then
        num_sent = num_sent + 1
    end
end

In [16]:
num_spaces = torch.DoubleTensor(num_sent,2)
row = 1
count_space = 0
for i=5,valid:size(1) do
    if valid[i] == 2 then
        num_spaces[{row, 1}] = row
        num_spaces[{row, 2}] = count_space
        count_space = 0
        row = row + 1
    elseif valid[i] == 1 then
        count_space = count_space + 1
    end
end

In [17]:
num_spaces[1]

 1
 7
[torch.DoubleTensor of size 2]



In [20]:
it = 1
i = 1
nextpred = torch.Tensor(2)

print('here')
while it<data['input_data_test']:size(1)-(Nwin-1) do
    if it % 50 == 0 then
        print( it..'/'..data['input_data_test']:size(1))
    end
    it = it + 1
    nextpred:copy(nnlm1:forward(test:narrow(1,i,Nwin)));
    m, argm = nextpred:max(1)

    if argm[1] == 2 then
        i = i + 1
    elseif argm[1] == 1 then 
        test_ = torch.LongTensor(test:size(1)+1)
        test_:narrow(1,1,i+(Nwin-1)):copy(test:narrow(1,1,i+(Nwin-1)))
        test_[i+Nwin] = 1
        test_:narrow(1,i+(Nwin-1)+2,test:size(1)-i-(Nwin-1)):copy(test:narrow(1,i+(Nwin-1)+1,test:size(1)-i-(Nwin-1)))
        test = test_
        i = i + 2
    end 
end

here	


50/367519	


100/367519	


150/367519	


200/367519	


...

367300/367519	


367350/367519	


367400/367519	


367450/367519	


367500/367519	


In [21]:
num_sent = 0
for i = 5,test:size(1) do
    if test[i] == 2 then
        num_sent = num_sent + 1
    end
end

print(num_sent)

3761	


In [22]:
num_spaces_test = torch.DoubleTensor(num_sent,2)
row = 1
count_space = 0
for i=5,test:size(1) do
    if test[i] == 2 then
        num_spaces_test[{row, 1}] = row
        num_spaces_test[{row, 2}] = count_space
        count_space = 0
        row = row + 1
    elseif test[i] == 1 then
        count_space = count_space + 1
    end
end

In [23]:
myFile = hdf5.open('../submission/pred_test_greedy_nn_5', 'w')
myFile:write('num_spaces', num_spaces_test)
myFile:close()